### Thinking1
在实际工作中，FM和MF哪个应用的更多，为什么

答：FM应用得越多，因为MF只是FM的一个特例，MF只利用了User-Item信息，即只对UserID和ItemID进行了特征embedding，只考虑了User和Item的交互关系，而FM可以更泛化地对其他信息进行特征embedding，利用MF考虑特征两两之间的交互关系，从而可以将更多的信息利用起来，同时解决了大规模稀疏矩阵的特征组合问题。除此之外，FM可以通过公式将预测时的计算复杂度降低到$O(k*n)$，所以FM在可以达到更高准确率的同时时间代价又在可接受范围内。

### Thinking2
FFM与FM有哪些区别？

答：FFM相比FM引入了filed的概念，即将one-hot编码前属于同一个字段的特征归属于同一个field，于是，原本FM中一个特征只对应一个隐变量，而在FFM中每一个特征都会有一组隐变量，其中一个隐变量表征该特征与一个field的交互关系，于是FFM的二次项$x_ix_j$的系数从FM中的$<V_i,V_j>$变为$<V_{i,f_j},V_{j,f_i}>$，其中$f_i,f_j$为$i,j$对应的field。这样的改变虽然使得模型有了更细致的刻画，但是由于更改之后预测时计算复杂度$O(k*n^2)$无法再使用公式来降低，选取隐变量维数$k$时，FFM通常远小于FM。

### Thinking3
DeepFM相比于FM解决了哪些问题，原理是怎样的

答：FM只考虑了1阶特征以及2阶特征的组合，但现实环境中，特征间的相互作用会更加复杂，但高阶特征的FM计算复杂度又过高，而DeepFM通过在FM的基础上加入DNN来做到能既考虑低阶特征又考虑高阶特征，其中FM负责低阶特征提取，DNN负责高阶特征提取，即：将1阶特征和特征embedding后的隐变量输入FM中进行学习的同时，将隐变量输入DNN来对更高阶特征进行学习，学习完成后，综合二者输出进行预测。

### Thinking4
Surprise工具中的baseline算法原理是怎样的？BaselineOnly和KNNBaseline有什么区别？

答：baseline的原理为：利用 $b_{ui}=\mu+b_u+b_i$ 来对$r_{ui}$进行预测，其中$\mu$为系统整体的均值，$b_u$为用户对整体的偏差，$b_i$为商品对整体的偏差。  
&emsp;&emsp;BaselineOnly是用$b_{ui}$来预测评分，进而通过对预测评分进行排序，来得到TopN推荐，而KNNBaseline只是利用Baseline的方法训练$b_u$和$b_i$，得到$b_{ui}$作为偏差，以便在考虑邻居的打分时，将偏差减掉。

### Thinking5
基于邻域的协同过滤都有哪些算法，请简述原理

答：基于领域的协同过滤包括UserCF和ItemCF。  
&emsp;&emsp;UserCF的原理为：定义用户间的相似度，对于指定用户，向他推荐那些与他相似的用户喜欢的商品。  
&emsp;&emsp;ItemCF的原理为：定义物品间的相似度，对于指定用户，找到与他喜欢的商品相似的那些商品进行推荐。

### Action1
使用libfm工具对movielens进行评分预测，采用SGD优化算法

#### （1）转换成libfm格式

![转换成libfm格式](https://github.com/ruizhuwu/BI/blob/master/week5/1.png)

#### （2）使用libfm，优化方法为SGD

设定参数：学习率，正则化项，初始参数分布方差

![使用libfm，优化方法为SGD——包括学习率，正则化项，初始参数分布](https://github.com/ruizhuwu/BI/blob/master/week5/2.png)

#### （3）完成100次迭代，输出为out.txt

![完成100次迭代](https://github.com/ruizhuwu/BI/blob/master/week5/3.png)

### Action2
使用DeepFM对movielens进行评分预测

#### （1）对于movielens_sample

In [1]:
import pandas as pd

# 载入数据
data = pd.read_csv("movielens_sample.txt")

# 数据探索
data.head()

,user_id,movie_id,rating,timestamp,title,genres,gender,age,occupation,zip
0,3299,235,4,968035345,Ed Wood (1994),Comedy|Drama,F,25,4,19119
1,3630,3256,3,966536874,Patriot Games (1992),Action|Thriller,M,18,4,77005
2,517,105,4,976203603,"Bridges of Madison County, The (1995)",Drama|Romance,F,25,14,55408
3,785,2115,3,975430389,Indiana Jones and the Temple of Doom (1984),Action|Adventure,M,18,19,29307
4,5848,909,5,957782527,"Apartment, The (1960)",Comedy|Drama,M,50,20,20009


In [2]:
from sklearn.preprocessing import LabelEncoder

# 对特征标签进行编码
sparse_features = ["movie_id", "user_id", "gender", "age", "occupation", "zip"] # 将年龄作为离散值处理
target = ['rating']
for feature in sparse_features:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature])
    
data.head()

,user_id,movie_id,rating,timestamp,title,genres,gender,age,occupation,zip
0,107,12,4,968035345,Ed Wood (1994),Comedy|Drama,0,2,4,35
1,123,169,3,966536874,Patriot Games (1992),Action|Thriller,1,1,4,118
2,12,6,4,976203603,"Bridges of Madison County, The (1995)",Drama|Romance,0,2,13,99
3,21,112,3,975430389,Indiana Jones and the Temple of Doom (1984),Action|Adventure,1,1,18,55
4,187,45,5,957782527,"Apartment, The (1960)",Comedy|Drama,1,5,19,41


In [3]:
from deepctr.feature_column import SparseFeat,get_feature_names

# 记录字段名,每个稀疏特征的不同值个数，以及embedding维数等
fixlen_feature_columns = [SparseFeat(feature, data[feature].nunique(),embedding_dim=8) for feature in sparse_features]
fixlen_feature_columns[0]

SparseFeat(name='movie_id', vocabulary_size=187, embedding_dim=8, use_hash=False, dtype='int32', embeddings_initializer=<tensorflow.python.keras.initializers.initializers_v1.RandomNormal object at 0x000001B5010D4910>, embedding_name='movie_id', group_name='default_group', trainable=True)

In [4]:
# 指定FM和DNN训练用的特征列，并获取特证名
dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

feature_names = get_feature_names(dnn_feature_columns + linear_feature_columns)
feature_names

['movie_id', 'user_id', 'gender', 'age', 'occupation', 'zip']

In [5]:
from sklearn.model_selection import train_test_split
from deepctr.models import DeepFM

# 切分数据集
train, test = train_test_split(data, test_size=0.2)

# 转换成字典
train_model_input = {name:train[name].values for name in feature_names}
test_model_input = {name:test[name].values for name in feature_names}

# 载入模型
model = DeepFM(linear_feature_columns,dnn_feature_columns,task='regression')
model.compile("adam", "mse", metrics=['mse'], )

# fit and predict
history = model.fit(train_model_input, train[target].values,
                    batch_size=256, epochs=10, verbose=1, validation_split=0.2, )
pred_ans = model.predict(test_model_input, batch_size=256)

Epoch 1/10


C:\Users\80566\Coding\anaconda3\lib\site-packages\tensorflow\python\framework\indexed_slices.py:431: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


1/1 [==============================] - 0s 275ms/step - loss: 14.1715 - mse: 14.1715 - val_loss: 14.1827 - val_mse: 14.1827
Epoch 2/10
1/1 [==============================] - 0s 15ms/step - loss: 14.0193 - mse: 14.0193 - val_loss: 14.0451 - val_mse: 14.0451
Epoch 3/10
1/1 [==============================] - 0s 15ms/step - loss: 13.8545 - mse: 13.8545 - val_loss: 13.8953 - val_mse: 13.8953
Epoch 4/10
1/1 [==============================] - 0s 15ms/step - loss: 13.6744 - mse: 13.6744 - val_loss: 13.7337 - val_mse: 13.7337
Epoch 5/10
1/1 [==============================] - 0s 14ms/step - loss: 13.4798 - mse: 13.4798 - val_loss: 13.5599 - val_mse: 13.5599
Epoch 6/10
1/1 [==============================] - 0s 15ms/step - loss: 13.2695 - mse: 13.2695 - val_loss: 13.3730 - val_mse: 13.3730
Epoch 7/10
1/1 [==============================] - 0s 15ms/step - loss: 13.0424 - mse: 13.0424 - val_loss: 13.1720 - val_mse: 13.1720
Epoch 8/10
1/1 [==============================] - 0s 15ms/step - loss: 12.7972 

In [6]:
# 计算RMSE
from sklearn.metrics import mean_squared_error
mse = round(mean_squared_error(test[target].values, pred_ans), 4)
rmse = mse ** 0.5
print("test RMSE:", rmse)

test RMSE: 3.4827431716967014


#### （2）对于movielens

In [7]:
import pandas as pd

# 载入数据
data = pd.read_csv("ratings.csv")

# 数据探索
data.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


In [8]:
from sklearn.preprocessing import LabelEncoder

# 对特征标签进行编码
sparse_features = ["userId", "movieId", "timestamp"] # 将年龄作为离散值处理
target = ['rating']
for feature in sparse_features:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature])
    
data.head()

,userId,movieId,rating,timestamp
0,0,1,3.5,340880
1,0,28,3.5,340785
2,0,31,3.5,340801
3,0,46,3.5,340790
4,0,49,3.5,340774


In [9]:
from deepctr.feature_column import SparseFeat,get_feature_names

# 记录字段名,每个稀疏特征的不同值个数，以及embedding维数等
fixlen_feature_columns = [SparseFeat(feature, data[feature].nunique()) for feature in sparse_features]
fixlen_feature_columns[0]

SparseFeat(name='userId', vocabulary_size=7120, embedding_dim=4, use_hash=False, dtype='int32', embeddings_initializer=<tensorflow.python.keras.initializers.initializers_v1.RandomNormal object at 0x000001B57F0459D0>, embedding_name='userId', group_name='default_group', trainable=True)

In [10]:
# 指定FM和DNN训练用的特征列，并获取特证名
dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

feature_names = get_feature_names(dnn_feature_columns + linear_feature_columns)
feature_names

['userId', 'movieId', 'timestamp']

In [11]:
from sklearn.model_selection import train_test_split
from deepctr.models import DeepFM
# 切分数据集
train, test = train_test_split(data, test_size=0.2)

# 转换成字典
train_model_input = {name:train[name].values for name in feature_names}
test_model_input = {name:test[name].values for name in feature_names}

# 载入模型
model = DeepFM(linear_feature_columns,dnn_feature_columns,task='regression')
model.compile("adam", "mse", metrics=['mse'], )

# fit and predict
history = model.fit(train_model_input, train[target].values,
                    batch_size=32, epochs=5, verbose=1, validation_split=0.2, )
pred_ans = model.predict(test_model_input, batch_size=256)

Epoch 1/5


C:\Users\80566\Coding\anaconda3\lib\site-packages\tensorflow\python\framework\indexed_slices.py:431: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


20972/20972 [==============================] - 646s 31ms/step - loss: 0.8225 - mse: 0.8138 - val_loss: 0.7541 - val_mse: 0.7411
Epoch 2/5
20972/20972 [==============================] - 644s 31ms/step - loss: 0.7066 - mse: 0.6892 - val_loss: 0.7415 - val_mse: 0.7218
Epoch 3/5
20972/20972 [==============================] - 658s 31ms/step - loss: 0.6478 - mse: 0.6267 - val_loss: 0.7373 - val_mse: 0.7144
Epoch 4/5
20972/20972 [==============================] - 642s 31ms/step - loss: 0.6222 - mse: 0.5983 - val_loss: 0.7412 - val_mse: 0.7151
Epoch 5/5
20972/20972 [==============================] - 631s 30ms/step - loss: 0.6164 - mse: 0.5907 - val_loss: 0.7367 - val_mse: 0.7095


In [12]:
# 计算RMSE
from sklearn.metrics import mean_squared_error
mse = round(mean_squared_error(test[target].values, pred_ans), 4)
rmse = mse ** 0.5
print("test RMSE:", rmse)

test RMSE: 0.8460496439334987


### Action3
使用基于邻域的协同过滤（KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline中的任意一种）对MovieLens数据集进行协同过滤，采用k折交叉验证(k=3)，输出每次计算的RMSE, MAE

In [14]:
import pandas as pd
from surprise import Reader
from surprise import Dataset
from surprise import KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline
from surprise.model_selection import cross_validate

# 载入数据
reader = Reader(line_format = 'user item rating timestamp', sep=',', skip_lines=1)
data = Dataset.load_from_file('ratings.csv', reader=reader)

# 数据探索
pd_data = pd.read_csv('ratings.csv')
print(pd_data.head())
print('用户数：', len(pd_data['userId'].unique()))
print('电影数：', len(pd_data['movieId'].unique()))

   userId  movieId  rating   timestamp
0       1        2     3.5  1112486027
1       1       29     3.5  1112484676
2       1       32     3.5  1112484819
3       1       47     3.5  1112484727
4       1       50     3.5  1112484580
用户数： 7120
电影数： 14026


In [15]:
# KNNBasic(因为电影数大于用户数，使用UserCF, 并设置最小支持度进行过滤)
sim_options = {'user_based': True, 'min_support': 10}
knnbasic = KNNBasic(sim_options=sim_options)

#k折交叉验证（k=3）
print('对于KNNBasic：')
cross_validate(knnbasic, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

对于KNNBasic：
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.8969  0.8960  0.8971  0.8967  0.0005  
MAE (testset)     0.6852  0.6842  0.6855  0.6850  0.0005  
Fit time          21.86   22.10   22.13   22.03   0.12    
Test time         118.59  117.12  118.68  118.13  0.71    


{'test_rmse': array([0.89690243, 0.8960176 , 0.8971442 ]),
 'test_mae': array([0.68515711, 0.68424551, 0.68554341]),
 'fit_time': (21.86299729347229, 22.09784770011902, 22.12903881072998),
 'test_time': (118.58775663375854, 117.12284564971924, 118.68268179893494)}

In [16]:
# KNNBasicWithMeans
sim_options = {'user_based': True, 'min_support': 10}
knnm = KNNWithMeans(sim_options=sim_options)

#k折交叉验证（k=3）
print('对于KNNBasicWithMeans：')
cross_validate(knnm, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

对于KNNBasicWithMeans：
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNWithMeans on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.8756  0.8749  0.8759  0.8755  0.0004  
MAE (testset)     0.6717  0.6719  0.6723  0.6720  0.0002  
Fit time          22.37   22.60   23.07   22.68   0.29    
Test time         125.11  127.34  126.04  126.17  0.91    


{'test_rmse': array([0.87564808, 0.87494866, 0.87585519]),
 'test_mae': array([0.67173312, 0.67194726, 0.67228126]),
 'fit_time': (22.36655902862549, 22.59949254989624, 23.066863536834717),
 'test_time': (125.11420941352844, 127.34312105178833, 126.04454708099365)}

In [17]:
# KNNBasicWithZScore
sim_options = {'user_based': True, 'min_support': 10}
knnz = KNNWithZScore(sim_options=sim_options)

#k折交叉验证（k=3）
print('对于KNNBasicWithZScore：')
cross_validate(knnz, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

对于KNNBasicWithZScore：
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNWithZScore on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.8738  0.8760  0.8744  0.8747  0.0009  
MAE (testset)     0.6658  0.6660  0.6657  0.6658  0.0001  
Fit time          23.03   24.49   23.21   23.58   0.65    
Test time         129.61  130.24  128.03  129.29  0.93    


{'test_rmse': array([0.87378473, 0.87602324, 0.87441383]),
 'test_mae': array([0.66579187, 0.66599656, 0.66573557]),
 'fit_time': (23.0324387550354, 24.49492621421814, 23.206647157669067),
 'test_time': (129.61101770401, 130.2395157814026, 128.03060841560364)}

In [19]:
# KNNBaseline(推荐使用皮尔森相似度)
sim_options = {'name': 'pearson_baseline', 'user_based': True, 'min_support': 10}
bsl_options = {'nethod': 'sgd', 'learning_rate': 0.0001}

knnbaseline = KNNBaseline(sim_options=sim_options, bsl_options=bsl_options)

#k折交叉验证（k=3）
print('对于KNNBaseline：')
cross_validate(knnbaseline, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

对于KNNBaseline：
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBaseline on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.8428  0.8452  0.8424  0.8435  0.0012  
MAE (testset)     0.6411  0.6434  0.6411  0.6419  0.0011  
Fit time          52.48   51.75   51.82   52.02   0.33    
Test time         132.40  130.54  133.48  132.14  1.22    


{'test_rmse': array([0.842807  , 0.84524622, 0.84244636]),
 'test_mae': array([0.64110371, 0.64344186, 0.64108161]),
 'fit_time': (52.48058819770813, 51.74988675117493, 51.82303738594055),
 'test_time': (132.40468621253967, 130.53682374954224, 133.48450469970703)}

&emsp;&emsp;以上结果与算法原理相符合，KNNWithMeans和KNNWithZScore效果差不多，均好于KNNBasic，KNNBaseline耗时最长，效果也最好，说明用机器拟合的偏差的确要好于均值、方差等人工设定的偏差。